- **Which datasets do you anticipate using in your final project? Are there columns that can be used to merge these datasets into a single dataframe?**

The datasets we plan to use in our final project are the section 12 alcohol licenses dataset and crime incidents data report from boston.gov. As well as the BARI census block groups mapping from Harvard dataverse and potentially the 911 calls ecometrics. The columns that can be used to merge these datasets are based on location. The crime and alcohol datasets include the address and street name of the business or incident. They also include latitude and longitude coordinates which are likely going to be most effective to use to find their position on the map. Since the geographic data reported by the Boston police can be slightly off we can't rely too heavily on it's accuracy, therefore when merging the data together with a spatial join it's important to note that these areas of city blocks represent overall crime in this area, but not at any exact address or coordinate. Other shared columns are the dates. The licenses and crime incidents both include the date of issue or occurrence which represents another possible area to merge the data although this is not our main focus for now.


- **What is the unit(s) of analysis you will be studying for your final project? In other words, what is the unit that each row in your final dataframe(s) will represent? Is this the same level of analysis that is represented in your raw data? If not, what columns in your raw data could be used to aggregate it to the level you need for your study?**

The unit of analysis for our project are city issued alcohol licenses in Boston, MA and incidents of alcohol related crimes. This is because we are looking to see if a neighborhood block group with a higher number of alcohol licenses is more prone to alcohol related crimes (Operating Under the Influence, Liquor Violation, and potentially violence related crimes based on peer reviewed research) than neighborhood block groups with fewer total licenses. In the final dataframe we would have latitude and longitude cordinates of these specific filtered crimes which are mapped onto specific city block groups in the greater Boston area. The level of analysis is not the same as represented in the raw data since our raw data includes the licenses and incidents on an individual basis. As stated above, we will need to do some type of aggregation (most likely spatial join) of crimes and licenses based on location from the raw data, in order to evaluate our research question.

### Load in and compile two datasets to find missing items for final question

In [1]:
import pandas as pd
import geopandas as gpd

In [7]:
crime_2015 = pd.read_csv('data/crime-incident-reports-2015.csv')
crime_2016 = pd.read_csv('data/crime-incident-reports-2015.csv')
crime_2017 = pd.read_csv('data/crime-incident-reports-2017.csv')
crime_2018 = pd.read_csv('data/crime-incident-reports-2018.csv')
crime = pd.DataFrame()
crime=crime.append(crime_2015, ignore_index=True)
crime=crime.append(crime_2016, ignore_index=True)
crime=crime.append(crime_2017, ignore_index=True)
crime=crime.append(crime_2018, ignore_index=True)
crime

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I192068249,2647,Other,THREATS TO DO BODILY HARM,B2,280,NaN,2015-08-28 10:20:00,2015,8,Friday,10,Part Two,WASHINGTON ST,42.330119,-71.084251,"(42.33011862, -71.08425106)"
1,I192061894,1106,Confidence Games,FRAUD - CREDIT CARD / ATM FRAUD,C11,356,NaN,2015-08-20 00:00:00,2015,8,Thursday,0,Part Two,CHARLES ST,42.300605,-71.061268,"(42.30060543, -71.06126785)"
2,I192038828,1107,Fraud,FRAUD - IMPERSONATION,A1,172,NaN,2015-11-02 12:24:00,2015,11,Monday,12,Part Two,ALBANY ST,42.334288,-71.072395,"(42.33428841, -71.07239518)"
3,I192008877,1107,Fraud,FRAUD - IMPERSONATION,E18,525,NaN,2015-07-31 10:00:00,2015,7,Friday,10,Part Two,WINGATE RD,42.237009,-71.129566,"(42.23700950, -71.12956606)"
4,I182090828,1102,Fraud,FRAUD - FALSE PRETENSE / SCHEME,D4,159,NaN,2015-12-01 12:00:00,2015,12,Tuesday,12,Part Two,UPTON ST,42.342432,-71.072258,"(42.34243222, -71.07225766)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307415,I070720870-00,802,Simple Assault,ASSAULT & BATTERY,B2,318,NaN,2018-12-13 00:00:00,2018,12,Thursday,0,Part Two,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
307416,I070720870-00,3125,Warrant Arrests,WARRANT ARREST,B2,318,NaN,2018-12-13 00:00:00,2018,12,Thursday,0,Part Three,BROOKLEDGE ST,42.309563,-71.089902,"(42.30956305, -71.08990197)"
307417,I060168073-00,1864,Drug Violation,DRUGS - POSS CLASS D - INTENT MFR DIST DISP,E13,912,NaN,2018-01-27 14:01:00,2018,1,Saturday,14,Part Two,CENTRE ST,42.322838,-71.100967,"(42.32283759, -71.10096723)"
307418,I060168073-00,1864,Drug Violation,DRUGS - POSS CLASS D - INTENT MFR DIST DISP,E13,912,NaN,2018-01-27 14:01:00,2018,1,Saturday,14,Part Two,CENTRE ST,42.322838,-71.100967,"(42.32283759, -71.10096723)"


In [6]:
alc = pd.read_csv('data/blb_section_12_alcohol.csv')

In [5]:
colleges = pd.read_csv('data/Colleges_and_Universities.csv')

In [13]:
off_codes = [2201, 2101, 2646]
crime = crime.loc[crime['OFFENSE_CODE'].isin(off_codes)]
crime

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
1066,I152107394,2101,Operating Under the Influence,OPERATING UNDER THE INFLUENCE ALCOHOL,B3,565,NaN,2015-12-30 00:55:00,2015,12,Wednesday,0,Part Two,HARVARD ST,42.282319,-71.099614,"(42.28231943, -71.09961428)"
1771,I152106611,2101,Operating Under the Influence,OPERATING UNDER THE INFLUENCE ALCOHOL,C11,353,NaN,2015-12-26 22:31:00,2015,12,Saturday,22,Part Two,WILLIAM T MORRISSEY BLVD,42.300270,-71.049671,"(42.30027016, -71.04967147)"
2059,I152106262,2101,Operating Under the Influence,OPERATING UNDER THE INFLUENCE ALCOHOL,C11,352,NaN,2015-12-25 07:26:00,2015,12,Friday,7,Part Two,ARCADIA ST,42.301415,-71.062878,"(42.30141461, -71.06287775)"
2075,I152106249,2101,Operating Under the Influence,OPERATING UNDER THE INFLUENCE ALCOHOL,A7,12,NaN,2015-12-25 05:02:00,2015,12,Friday,5,Part Two,SARATOGA ST,42.385047,-71.013741,"(42.38504650, -71.01374142)"
2091,I152106231,2101,Operating Under the Influence,OPERATING UNDER THE INFLUENCE ALCOHOL,A7,28,NaN,2015-12-25 02:41:00,2015,12,Friday,2,Part Two,CENTRAL SQ,42.374686,-71.038931,"(42.37468615, -71.03893111)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307095,I182000120,2646,Liquor Violation,LIQUOR - DRINKING IN PUBLIC,B2,280,NaN,2018-01-01 11:38:00,2018,1,Monday,11,Part Two,WASHINGTON ST,42.329259,-71.084824,"(42.32925905, -71.08482402)"
307107,I182000101,2101,Operating Under the Influence,OPERATING UNDER THE INFLUENCE ALCOHOL,C11,359,NaN,2018-01-01 07:58:00,2018,1,Monday,7,Part Two,REGINA RD,42.295028,-71.072040,"(42.29502823, -71.07203995)"
307134,I182000064,2101,Operating Under the Influence,OPERATING UNDER THE INFLUENCE ALCOHOL,B2,,NaN,2018-01-01 03:29:00,2018,1,Monday,3,Part Two,NaN,NaN,NaN,"(0.00000000, 0.00000000)"
307163,I182000032,2101,Operating Under the Influence,OPERATING UNDER THE INFLUENCE ALCOHOL,D4,146,NaN,2018-01-01 01:34:00,2018,1,Monday,1,Part Two,HUNTINGTON AVE,42.346659,-71.079980,"(42.34665908, -71.07997968)"


In [16]:
alc = alc.loc[alc['license_type'].str.contains('All Alc.')]
alc

,license_num,historicallicensenum,status,license_type,issued,expires,business_name,dba_name,comments,location_comments,...,applicant,manager,day_phone,evening_phone,address,city,state,zip,gpsx,gpsy
3,LB-410009,NaN,Active,CV7 All Alc. Airp.,2020-03-04,2020-12-31,"SSP America BOS, LLC",Wahlburgers,NaN,"3,033 sq. ft. restaurant located within Termi...",...,"SSP America BOS, LLC",Jacqueline Genao,(781)808-8354,NaN,Logan Airport 300 Terminal C,East Boston,MA,02128,0.000000,0.000000e+00
6,LB-414876,NaN,Active,CV7 All Alc. Airp.,2020-02-11,2020-12-31,"Delaware North Boston Flight, LLC",B Good Burger,NaN,"Restaurant concept in Terminal A, Space AM2-A0...",...,"Delaware North Boston Flight, LLC",Mark V. Clemency,NaN,NaN,100 Terminal RD,East Boston,MA,02128,785396.803968,2.958230e+06
8,LB-419273,NaN,Active,CV7 All Alc.,2020-01-07,2020-12-31,KJ's Irish Pub,KJ's,Bottle service only with Board BPD approval,In whole of said triple-decker building.,...,KJ's Irish Pub,Kathleen Joyce,NaN,NaN,15 N Munroe TER,Dorchester,MA,02122,777349.961472,2.932091e+06
9,LB-408105,NaN,Active,CV7 All Alc. Airp.,2019-12-19,2020-12-31,"Delaware North Boston Flight, LLC",Mija Cantina,NaN,"Restaurant/full bar concept in Terminal C, Spa...",...,"Delaware North Boston Flight, LLC",Mark V. Clemency,(716)828-5162,NaN,Logan Airport Terminal C,East Boston,MA,02128,0.000000,0.000000e+00
10,LB-400460,NaN,Active,CV7 All Alc. Airp.,2019-11-27,2020-12-31,"HSI MCA BOS FB, LLC",Santarpio's Pizza,NaN,"Space CB2-A55 in Terminal C, Pier B at Loagan ...",...,"HSI MCA BOS FB, LLC",SAIKOU BOIRO,NaN,NaN,Logan Airport Terminal C,East Boston,MA,02128,0.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1425,LB-98819,CLBALA0002,Active,Clb. All Alc. Airport,2013-10-01,2020-12-31,"AMERICAN AIRLINES, INC.",ADMIRALS CLUB,NaN,"Approximately 15,317 Sq. Ft. retail space with...",...,"AMERICAN AIRLINES, INC.",MICHELE FRENI,NaN,(781)944-1506,Logan Airport Terminal B,East Boston,MA,02128,NaN,NaN
1427,LB-414874,NaN,Active,CV7 All Alc. Airp.,NaN,NaN,"Delaware North Boston Flight, LLC",Alta Strada,NaN,"Restaurant/full bar concept in Terminal A, Spa...",...,"Delaware North Boston Flight, LLC",Mark V. Clemency,NaN,NaN,100 Terminal RD,East Boston,MA,02128,785396.803968,2.958230e+06
1429,LB-418841,NaN,Active,CV7 All Alc. Airp.,NaN,NaN,"Delaware North Boston Flight, LLC",North End Bar,NaN,"Restaurant/full bar concept in Terminal B, Spa...",...,"Delaware North Boston Flight, LLC",Mark V. Clemency,NaN,NaN,200 Terminal RD,East Boston,MA,02128,786260.691516,2.957556e+06
1432,LB-409585,NaN,Active,CV7 All Alc. Restrict.,NaN,NaN,"The Pearl, Seafood Grill and Raw Bar, Inc.",The Pearl,NaN,"On first floor: kitchen, bar, and dining area ...",...,"The Pearl, Seafood Grill and Raw Bar, Inc.",Mika Winder,(617)510-0267,NaN,20B District Av,Dorchester,MA,02125,774405.476887,2.943901e+06


In [4]:
nb = gpd.read_file("data/Boston_Neighborhoods.geojson")
nb

,OBJECTID,Name,Acres,Neighborhood_ID,SqMiles,ShapeSTArea,ShapeSTLength,geometry
0,27,Roslindale,1605.568237,15,2.51,6.993827e+07,53563.912597,"MULTIPOLYGON (((-71.12593 42.27201, -71.12611 ..."
1,28,Jamaica Plain,2519.245394,11,3.94,1.097379e+08,56349.937161,"POLYGON ((-71.10499 42.32610, -71.10503 42.326..."
2,29,Mission Hill,350.853564,13,0.55,1.528312e+07,17918.724113,"POLYGON ((-71.09043 42.33577, -71.09050 42.335..."
3,30,Longwood,188.611947,28,0.29,8.215904e+06,11908.757148,"POLYGON ((-71.09811 42.33673, -71.09832 42.337..."
4,31,Bay Village,26.539839,33,0.04,1.156071e+06,4650.635493,"POLYGON ((-71.06663 42.34878, -71.06663 42.348..."
5,32,Leather District,15.639908,27,0.02,6.812717e+05,3237.140537,"POLYGON ((-71.05838 42.34983, -71.05834 42.349..."
6,33,Chinatown,76.324410,26,0.12,3.324678e+06,9736.590413,"POLYGON ((-71.05791 42.35238, -71.05811 42.352..."
7,34,North End,126.910439,14,0.20,5.527506e+06,16177.826815,"POLYGON ((-71.05200 42.36884, -71.05219 42.368..."
8,35,Roxbury,2108.469072,16,3.29,9.184455e+07,49488.800485,"POLYGON ((-71.09646 42.29323, -71.09633 42.293..."
9,36,South End,471.535356,32,0.74,2.054000e+07,17912.333569,"POLYGON ((-71.06834 42.34775, -71.06913 42.347..."


# start with first crime csv
df_crime = pd.read_csv('data/2012-2015-crime.csv.gz')

# append all crime csv's into one dataframe
for year in range(2015, 2022):
    # form standard name convention for each year
    file_name = 'data/' + str(year) + '-crime.csv.gz'
    df_crime.append(pd.read_csv(file_name), ignore_index=True)

# license data is all in one
df_alc = pd.read_csv('data/alcohol.csv.gz')

df_alc

df_crime

# shows types of criminal offenses, we can use Liquor Violations and Operating Under the Influence
# (or other based on research linking crime and alcohol)
df_crime['OFFENSE_CODE_GROUP'].unique()

### Find percentage of missing data from key rows

print("Percentage of missing 'Lat' data in crime dataset:")
print(sum(df_crime.Lat.isna()) / len(df_crime)*100,'%')

print("Percentage of missing 'Long' data in crime dataset:")
print(sum(df_crime.Long.isna()) / len(df_crime)*100, '%')

print("Percentage of missing 'OFFENSE_CODE_GROUP' data in crime dataset:")
print(sum(df_crime.OFFENSE_CODE_GROUP.isna()) / len(df_crime)*100, '%')

print("Percentage of missing 'OFFENSE_CODE' data in crime dataset:")
print(sum(df_crime.OFFENSE_CODE.isna()) / len(df_crime)*100, '%')

print("Percentage of missing 'OCCURRED_ON_DATE' data in crime dataset:")
print(sum(df_crime.OCCURRED_ON_DATE.isna()) / len(df_crime)*100, '%')

print('-' * 50)

print("Percentage of missing 'gpsx' data in liquor license dataset:")
print(sum(df_alc.gpsx.isna()) / len(df_crime)*100, '%')

print("Percentage of missing 'gpsy' data in liquor license dataset:")
print(sum(df_alc.gpsy.isna()) / len(df_crime)*100, '%')

print("Percentage of missing 'ISSUED' data in liquor license dataset:")
print(sum(df_alc.issued.isna()) / len(df_crime)*100, '%')

print("Percentage of missing 'license_type' data in liquor license dataset:")
print(sum(df_alc.license_type.isna()) / len(df_crime)*100, '%')

- **If you have the data for your project in hand: Evaluate your main columns of interest for missing data. What percentage of rows are missing data? Can you see any patterns in which rows are missing data?**

The main columns of interest chosen in the evaluation above are the location data of both the licenses and crime incidents and various other important factors. The location data for both is split up into latitude and longitude coordinates where some rows have both values NA. In the crime dataset 3.97% of rows contain missing location data and in the licenses dataset 0.09% contained missing location data. Given the crime dataset contains nearly 35x more rows than the licenses, with ~4% of crimes missing a location we are still left with a large number of crime incidents (51,000+) with a location. The other important features for crime including the offense code and occurence date contain no NA values. The issued date column for the licenses contained 0.013% NA values which, similar to the location, is a very small portion of the data. The licenses type column contained 0% NA values.

A general pattern observed in the missing data is that latitude and longitude columns are missing the same data, which is helpful because it means that either the entire location was recorded or not at all. If it was not recorded we can exclude this point. Another note is that in the future we will need additional techniques beyond just checking NA for the location. We could also check if the location is outside of Boston, since we only want to include crimes in this area, and there could be rows with errors in the latitude and longitude that we don't want included in our analysis, but aren't NA.